In [1]:
import pandas
from io import StringIO
import paramiko
import os
import stat
import re

# Libraries Brian wants

In [84]:
brians_list = StringIO("""age     tissue  Library_number
e11.5   forebrain       15007
e11.5   forebrain       15008
e11.5   midbrain        15009
e11.5   midbrain        15010
e11.5   hindbrain       15011
e11.5   hindbrain       15012
e11.5   cranioface      15013
e11.5   cranioface      15014
e11.5   heart   15015
e11.5   heart   15016
e11.5   liver   15017
e11.5   liver   15018
e11.5   limb    15019
e11.5   limb    15020
e11.5   neural_tube     15021
e11.5   neural_tube     15022
e14.5   forebrain       15072
e14.5   forebrain       15073
e14.5   midbrain        15074
e14.5   midbrain        15075
e14.5   hindbrain       15076
e14.5   hindbrain       15077
e14.5   cranioface      15078
e14.5   cranioface      15079
e14.5   liver   15080
e14.5   liver   15081
e14.5   lung    15082
e14.5   lung    15083
e14.5   limb    15084
e14.5   limb    15085
e14.5   heart   15086
e14.5   heart   15087
e14.5   stomach 15088
e14.5   stomach 15089
e14.5   intestine       15090
e14.5   intestine       15091
e14.5   neural_tube     15092
e14.5   neural_tube     15093
e14.5   kidney  15094
e14.5   kidney  15095
P0      forebrain       15475
P0      forebrain       15476
P0      midbrain        15477
P0      midbrain        15478
P0      hindbrain       15479
P0      hindbrain       15480
P0      neural_tube     15481
P0      neural_tube     15482
P0      thymus  15483
P0      thymus  15484
P0      heart   15485
P0      heart   15486
P0      lungs   15487
P0      lungs   15488
P0      liver   15489
P0      liver   15490
P0      skeletal_muscle 15491
P0      skeletal_muscle 15492
P0      spleen  15493
P0      spleen  15494
P0      bladder 15495
P0      bladder 15496
P0      stomach 15497
P0      stomach 15498
P0      intestine       15499
P0      intestine       15500
P0      kidney  15501
P0      kidney  15502
P0      adrenals        15503
P0      adrenals        15504
""")
brians_df = pandas.read_csv(brians_list, sep=' *', index_col=2)
brians_df

/usr/lib/python3/dist-packages/pandas/io/parsers.py:639: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


,age,tissue
Library_number,,
15007,e11.5,forebrain
15008,e11.5,forebrain
15009,e11.5,midbrain
15010,e11.5,midbrain
15011,e11.5,hindbrain
15012,e11.5,hindbrain
15013,e11.5,cranioface
15014,e11.5,cranioface
15015,e11.5,heart


In [83]:
pandas.read_csv?

/usr/lib/python3/dist-packages/pandas/io/parsers.py:639: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


StopIteration: 

# SSH search

In [55]:
def walk(sftp, cwd=None):
    if cwd is None:
        cwd = sftp.getcwd()
    dirnames = []
    filenames = []
    
    for name in sftp.listdir():
        attrs = sftp.stat(os.path.join(cwd, name))
        if stat.S_ISDIR(attrs.st_mode):
            dirnames.append(name)
        else:
            filenames.append(name)

    yield (cwd, dirnames, filenames)
    
    for name in dirnames:
        nextdir = os.path.join(cwd, name)
        sftp.chdir(nextdir)
        yield from walk(sftp, nextdir)
    
    sftp.chdir(cwd)



In [26]:
def find_bigwigs(sftp):
    is_bigwig = re.compile('^(?P<libid>[\d]{5})-.*\.bigWig')
    available_bigwigs = {}
    for cwd, dirnames, filenames in walk(sftp):
        for filename in filenames:
            if filename == 'accepted_hits.bigWig':
                base, cur = os.path.split(cwd)
                libid = int(cur[:5])
                available_bigwigs[libid] = os.path.join(cwd, filename)
            else:
                # labeled bigiwg
                match = is_bigwig.match(filename)
                if match:
                    libid = match.group('libid')
                    available_bigwigs[libid] = os.path.join(cwd, filename)
    return available_bigwigs

In [6]:
host = paramiko.SSHClient()
host.set_missing_host_key_policy(paramiko.WarningPolicy())
host.connect('pongo.cacr.caltech.edu', username='diane')
sftp = host.open_sftp()


/usr/lib/python3/dist-packages/paramiko/client.py:580: UserWarning: Unknown ssh-rsa host key for pongo.cacr.caltech.edu: b'6833531597d6176b45c27f33e9406a12'
  (key.get_name(), hostname, hexlify(key.get_fingerprint())))


# Libraries run by georgi

In [62]:
#sftp.chdir('/woldlab/castor/proj/Single-Cell-RNA-seq/Data-Mouse-Embryos-mm10')
georgi_root = '/woldlab/castor/home/georgi/public_html/mouse-embryo-mm10-SMART'
sftp.chdir(georgi_root)
georgi_bigwigs = find_bigwigs(sftp)

In [30]:
sftp.chdir('/woldlab/castor/home/sau/public_html')
sau_bigwigs = find_bigwigs(sftp)

In [31]:
all_bigwigs = set(sau_bigwigs.keys()).union(georgi_bigwigs.keys())

In [29]:
georgi_bigwigs[15363]

'/woldlab/castor/proj/Single-Cell-RNA-seq/Data-Mouse-Embryos-mm10/15363-100mers-e11.5Fb1_SMARTpredNTP-TopHat-2.0.8-GENCODE-M3/accepted_hits.bigWig'

In [47]:
set(brians_df['Library_number']).difference(georgi_bigwigs)

set()

In [71]:
georgi_url = 'https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART'
def format_georgi(path):
    url = path.replace(georgi_root, georgi_url)
    _, description = os.path.split(path.replace('/accepted_hits.bigWig',''))
    return (url, description)

georgi_df = pandas.DataFrame(
    [ format_georgi(georgi_bigwigs[k]) for k in georgi_bigwigs],
    index=georgi_bigwigs.keys(),
    columns=['url', 'description'])
georgi_df

,url,description
15488,https://woldlab.caltech.edu/~georgi/mouse-embr...,15488-100mers-P0_lungs2_10ngs-TopHat-2.0.8-GEN...
15489,https://woldlab.caltech.edu/~georgi/mouse-embr...,15489-100mers-P0_liver1_10ngs-TopHat-2.0.8-GEN...
15490,https://woldlab.caltech.edu/~georgi/mouse-embr...,15490-100mers-P0_liver2_10ngs-TopHat-2.0.8-GEN...
15491,https://woldlab.caltech.edu/~georgi/mouse-embr...,15491-100mers-P0_skeletal_muscle1_10ngs-TopHat...
15492,https://woldlab.caltech.edu/~georgi/mouse-embr...,15492-100mers-P0_skeletal_muscle2_10ngs-TopHat...
15493,https://woldlab.caltech.edu/~georgi/mouse-embr...,15493-100mers-P0_spleen1_10ngs-TopHat-2.0.8-GE...
15494,https://woldlab.caltech.edu/~georgi/mouse-embr...,15494-100mers-P0_spleen2_10ngs-TopHat-2.0.8-GE...
15495,https://woldlab.caltech.edu/~georgi/mouse-embr...,15495-100mers-P0_bladder1_10ngs-TopHat-2.0.8-G...
15496,https://woldlab.caltech.edu/~georgi/mouse-embr...,15496-100mers-P0_bladder2_10ngs-TopHat-2.0.8-G...
15497,https://woldlab.caltech.edu/~georgi/mouse-embr...,15497-100mers-P0_stomach1_10ng-TopHat-2.0.8-GE...


In [86]:
trackinfo = pandas.concat([brians_df, georgi_df], axis=1)

In [97]:
track = "track type=bigWig name={name} description={description} color=255,0,0 bigDataUrl={url}"
for key in trackinfo.index:
    row = trackinfo.ix[key]
    print(track.format(name=key, **row))

track type=bigWig name=15007 description=15007-100mers-E11.5-forebrain1_10ngs-TopHat-2.0.8-GENCODE-M3 color=255,0,0 bigDataUrl=https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART/15007-100mers-E11.5-forebrain1_10ngs-TopHat-2.0.8-GENCODE-M3/accepted_hits.bigWig
track type=bigWig name=15008 description=15008-100mers-E11.5-forebrain2_10ngs-TopHat-2.0.8-GENCODE-M3 color=255,0,0 bigDataUrl=https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART/15008-100mers-E11.5-forebrain2_10ngs-TopHat-2.0.8-GENCODE-M3/accepted_hits.bigWig
track type=bigWig name=15009 description=15009-100mers-E11.5-midbrain1_10ngs-TopHat-2.0.8-GENCODE-M3 color=255,0,0 bigDataUrl=https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART/15009-100mers-E11.5-midbrain1_10ngs-TopHat-2.0.8-GENCODE-M3/accepted_hits.bigWig
track type=bigWig name=15010 description=15010-100mers-E11.5-midbrain2_10ngs-TopHat-2.0.8-GENCODE-M3 color=255,0,0 bigDataUrl=https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART/15010-100

In [96]:
dict(trackinfo.ix[15007])

{'age': 'e11.5',
 'description': '15007-100mers-E11.5-forebrain1_10ngs-TopHat-2.0.8-GENCODE-M3',
 'tissue': 'forebrain',
 'url': 'https://woldlab.caltech.edu/~georgi/mouse-embryo-mm10-SMART/15007-100mers-E11.5-forebrain1_10ngs-TopHat-2.0.8-GENCODE-M3/accepted_hits.bigWig'}